In [26]:
import mysql.connector
from mysql.connector import errorcode
import pandas as pd
from mysql.connector import IntegrityError

In [27]:
try:
    # Intenta hacer la conexión con la base de datos 
    cnx = mysql.connector.connect(
        user='root',
        password='AlumnaAdalab',
        host='127.0.0.1',
        database='music_stream'
    )
    print("¡Conexión exitosa a MySQL!")
except mysql.connector.Error as err:
    # Si es un error de acceso denegado (ej. contraseña o usuario incorrecto)
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Algo está mal con tu nombre de usuario o contraseña.") #
    # Si la base de datos no existe
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("La base de datos no existe.") #
    # Para cualquier otro tipo de error
    else:
        print(err) #
        print("Código de Error:", err.errno) #
        print("SQLSTATE", err.sqlstate) #
        print("Mensaje", err.msg)

¡Conexión exitosa a MySQL!


In [28]:
mycursor = cnx.cursor() # Iniciamos el cursor

mycursor.execute("SHOW TABLES") # Ejecutamos la query

# El cursor se convierte en un iterable al que podemos acceder, por ejemplo, con un bucle for
print("Tablas en la base de datos 'tienda':")
for x in mycursor:
    print(x) # Devuelve cada resultado como una tupla

Tablas en la base de datos 'tienda':
('artist',)
('artist_genre',)
('genre',)
('similar_artist',)
('track',)


In [ ]:
# con esto subimos únicamente el nombre de los artistas desde los csv específicos de artistas.
# es una primera prueba y también una forma de tener ya la PK name_artist para luego poder usarla
# como FK en la subida de tracks, que es la más "sencilla" para una primera prueba

# Cargar el CSV en un DataFrame
csv = 'hiphop_artistas.csv' 
data = pd.read_csv(csv)

# iterar en el dataframe e insertar en la tabla de MySQL
for index, row in data.iterrows():
    try:
        mycursor.execute(
            "INSERT INTO artist (name_artist) VALUES (%s)",
            (row['0'],)
        )
    except IntegrityError as e:
        if e.errno == 1062:
            print(f"Registro duplicado ignorado: {row['0']}")
        else:
            print(f"Error en fila {index}: {e}")

# hacer el commit
cnx.commit()

print("Datos importados")

In [29]:
# aquí subimos toda la información de los tracks

# Cargar el CSV en un DataFrame
csv = 'indie_resultados.csv' 
data = pd.read_csv(csv)

# ha estado dando problemas con valores NaN, esto es un intento de solucionarlo
# con fillna lo que hacemos es sustituir valores NaN, que esto no lo interpreta,
# por, en este caso, un 0
data.fillna(0, inplace=True)

# iterar en el dataframe e insertar en la tabla de MySQL
for index, row in data.iterrows():
   mycursor.execute(
# aquí, como teníamos ya un número asignado a la id, metemos que haga update para 
# que no nos dé error por intentar llenar una columna que ya tiene información
        """
    INSERT INTO track (name_artist, type, name, year, track_id)
    VALUES (%s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
        name_artist = VALUES(name_artist),
        type = VALUES(type),
        name = VALUES(name),
        year = VALUES(year)
    """,
        (row['artista'], row['tipo'], row['nombre'], row['anio_lanzamiento'], row['id'])
    )

# hacer el commit
cnx.commit()

print("Datos importados")

# Cerrar la conexión
mycursor.close()
cnx.close()

Datos importados
